In [1]:
# Mel Frequency Cepstral Coefficients
from python_speech_features import mfcc
import scipy.io.wavfile as wav      # Mengembalikan sample rate dan data dari file .wav
import numpy as np

import os
import pickle                              # Library untuk mengimplementasikan protokol biner untuk membuat serialisasi dan de-serialisasi struktur objek Python.
import random                          # Library untuk mengimplementasikan generator bilangan pseudo-random untuk berbagai distribusi.
import operator

In [2]:
# Untuk mendapatkan jarak antara vektor fitur dan menemukan neighbors
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [3]:
# Untuk Mengidentifikasi kelas instance
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [4]:
# Function untuk mengevaluasi model
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    
    return (1.0 * correct) / len(testSet)

In [5]:
# Letak direktori yang menyimpan file ekstensi .wav
directory = "/MATA KULIAH/Semester 7/DATA MINING (DAMI)/PROYEK BESAR/IMPLEMENTASI/genres/"

# file biner tempat semua fitur akan dikumpulkan yang diekstrak menggunakan mfcc (Koefisien Cepstral Frekuensi Mel)
f = open("music.dat", 'wb')

i = 0

for folder in os.listdir(directory):
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+folder):        
        try:
            (rate, sig) = wav.read(directory+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print('Got an exception: ', e, ' in folder: ', folder, ' filename: ', file)        

f.close()

In [6]:
# Membagi set data menjadi training set dan testing set masing-masing
dataset = []

def loadDataset(filename, split, trSet, teSet):
    with open('music.dat', 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])
trainingSet = []
testSet = []
loadDataset('music.dat', 0.66, trainingSet, testSet)

In [7]:
# Function untuk melakukan perhitungan jarak aktual antar features
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

In [ ]:
# Membuat prediksi dengan menggunakan KNN
leng = len(testSet)
predictions = []
for x in range(leng):
    predictions.append(nearestClass(getNeighbors(trainingSet, testSet[x], 5)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

In [ ]:
# Melakukan pengujian kode dengan memasukkan file audio
# File audio berekstensi .wav

test_dir = "/MATA KULIAH/Semester 7/DATA MINING (DAMI)/PROYEK BESAR/IMPLEMENTASI/Test/"
test_file = test_dir + "test3.wav" #Clasiccal
#test_file = test_dir + "test2.wav"
# test_file = test_dir + "test4.wav"


In [ ]:
(rate, sig) = wav.read(test_file)
mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix, covariance, i)

In [ ]:
from collections import defaultdict
results = defaultdict(int)

directory = "/MATA KULIAH/Semester 7/DATA MINING (DAMI)/PROYEK BESAR/IMPLEMENTASI/genres/"

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [ ]:
pred = nearestClass(getNeighbors(dataset, feature, 5))
print(results[pred])